In [1]:
!python --version

Python 3.7.6


In [2]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 1.13.1 requires tensorboard<1.14.0,>=1.13.0, but you have tensorboard 1.14.0 which is incompatible.
tensorflow-gpu 1.13.1 requires tensorflow-estimator<1.14.0rc0,>=1.13.0, but you have tensorflow-estimator 1.14.0 which is incompatible.


In [120]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [2]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [3]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [15]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [16]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [121]:
ws = WS('./data/')
pos = POS('./data/')

In [28]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

  0%|          | 0/1800 [00:00<?, ?it/s]

In [32]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [41]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
index = 0
for i, l in tqdm(train_title_cuts.items()):
    for word, tag in l:
        if word not in word2index:
            word2index[word] = index
            index2word[index] = word
            index += 1 
# END YOUR CODE

  0%|          | 0/1800 [00:00<?, ?it/s]

In [43]:
word2index['溫暖']

1520

In [44]:
index2word[1520]

'溫暖'

In [46]:
len(word2index)

6690

In [49]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for word, tag in pairs:
        vector[word2index[word]] += 1
    # END YOUR CODE
    return vector

In [16]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [51]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['勒布朗', '巴菲特', '雪迪龍', '亞亞·圖雷', '恆生', '巴薩', '小強', '庫里', '堪比比薩', 'U23', '雷軍', '羅永浩', '章澤天', '勇士隊', '林員', '微眾', '賞月夜', '韓長賦', '人民日報', '裴擒虎', '宏運', '塔特姆', '維亞利', '密子君', '高傭', '趙雲', '張某', '黃希揚', '西游', '碧生源', '尤文', '伏爾加格勒', '綠心', 'NBA', '旭旭', '亨德森', '劉歡', '林美景', '德甲', '意大利杯', '泰拳王西', '馬雲', '萊萬特', '鎧單', '后羿', '特里', '巔峰賽', '侯森', '拳皇', '安卓', '傑尼亞', '馬夏爾', '海富通', '小詹皇', '樊振東', '溫格', '海因克斯', '波爾', '杜蘭特', '韓信', '慧商', '王者榮耀', '李白', '招行', '亞特拉斯賽', '姆巴佩', '周雲傑', '雲聯惠', '奧恰洛夫', '哈希', '何雯娜', '盧克肖', '微博', '加威廉', '朱婷', '伯克希爾', '曼聯', '峽谷里', '雅桑克萊', '拜仁續', '猛', '李', '游玄德', '李秋平', '重渡溝', '董明珠', '張', '林斌', '康德', '李大霄', '游古', '小明', '纏中', '世界杯', '樂福', '上港', '洛瑞', '騎士隊', '梅西', '拉基蒂奇']
VC : ['拿', '掌握', '耍', '計算', '治療', '殺', '看直', '看', '收穫', '撿', '實拍', '簽中', '簽', '存', '贊取', '套種', '審查', '防守', '參透', '對付', '詮釋', '增持', '賣出', '承受', '養殖', '分析', '殺進', '拉開', '管理', '打飛', '禁', '衝施', '分銷', '體驗', '化解', '興起', '推出', '防治', '說動', '帶上', '暢玩', '分', '決殺', '纏', '玩上', '保駕', '零售', '選', '採用', '洩露', '生產', '打到', '調侃', '攻', '顛覆', '盯'

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [126]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'Caa', 'Cab', 'Cba', 'Cbb', 'I', 'Nf', 'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY',
        'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY', 'PARENTHESISCATEGORY', 'PAUSECATEGORY', 
        'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY', 'SPCHANGECATEGORY', 'WHITESPACE'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [127]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    similarity = np.dot(bow1, bow2)/(np.linalg.norm(bow1) * np.linalg.norm(bow2))
    # END YOUR CODE
    return similarity

In [128]:
bow1 = get_bow_vector_with_selection(train_title_cuts[100], word2index)
bow2 = get_bow_vector_with_selection(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.0

In [129]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [130]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [131]:
train_title_cuts[0]

[('亞洲杯', 'Nb'),
 ('奪', 'VC'),
 ('冠賠率', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日本', 'Nc'),
 ('、', 'PAUSECATEGORY'),
 ('伊朗', 'Nc'),
 ('領銜', 'VB'),
 (' ', 'WHITESPACE'),
 ('中國', 'Nc'),
 ('竟', 'D'),
 ('與', 'P'),
 ('泰國', 'Nc'),
 ('並列', 'VG')]

In [132]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [133]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

  0%|          | 0/600 [00:00<?, ?it/s]

In [134]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [135]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 65, '旅遊': 11, '遊戲': 9, '財經': 6, '農業': 5, '科技': 3})
predict 財經 :  Counter({'財經': 66, '科技': 19, '農業': 9, '體育': 8, '旅遊': 8, '遊戲': 5})
predict 科技 :  Counter({'科技': 60, '農業': 16, '體育': 13, '財經': 13, '旅遊': 8, '遊戲': 5})
predict 旅遊 :  Counter({'旅遊': 55, '農業': 11, '財經': 9, '體育': 3, '科技': 3, '遊戲': 2})
predict 農業 :  Counter({'農業': 56, '科技': 6, '旅遊': 5, '遊戲': 4, '體育': 2, '財經': 2})
predict 遊戲 :  Counter({'遊戲': 75, '旅遊': 12, '體育': 8, '科技': 8, '財經': 4, '農業': 3})
